In [1]:
from kafka import KafkaConsumer
import json
import pandas as pd
from sqlalchemy import create_engine, text
import time

In [2]:
#Destination config
username = 'sa'
password = '1'
hostname = 'DESKTOP-6BJG2TB'
port = '1433'
database_destination = 'DWH'
table_name = 'CustomerDetails' #Check data from what table?
database_source = 'Customers'

In [3]:
#read json data to dataframe
def read_json_to_df(json):
    data = pd.DataFrame([json])
    return data

In [4]:
def insert_data(df):
    #Take new data
    cdc_table = pd.DataFrame([df.loc[0,'after']]) 
    #engine connect to mssql
    engine = create_engine(f'mssql+pymssql://{username}:{password}@{hostname}:{port}/{database_destination}') 
    
    print(f"Add ID {cdc_table.loc[0,'id']} to {table_name}")

    cdc_table.to_sql(f'{table_name}', engine, if_exists='append', index = False)

In [5]:
def update_data(df):
    cdc_table = pd.DataFrame([df.loc[0,'after']])
    engine = create_engine(f'mssql+pymssql://{username}:{password}@{hostname}:{port}/{database_destination}')
    #string set by loop
    set_str = 'SET '
    for col in cdc_table.columns:
        set_str = set_str + f' {col} = {cdc_table.loc[0,'col']},'
    #SQL command
    sql = f"UPDATE {table_name}" + set_str[:-1] + f" WHERE {table_name}.id = {cdc_table.loc[0,'id']}"
    
    print(f"Update ID {cdc_table.loc[0,'id']} from {table_name}")

    #run sql command
    with engine.begin() as conn:
        conn.execute(text(sql))

In [6]:
def delete_data(df):
    cdc_table = pd.DataFrame([df.loc[0,'before']])
    engine = create_engine(f'mssql+pymssql://{username}:{password}@{hostname}:{port}/{database_destination}')

    sql = f"DELETE FROM {table_name} WHERE {table_name}.id = {cdc_table.loc[0,'id']}"
    print(f"Delete ID {cdc_table.loc[0,'id']} from {table_name}")

    with engine.begin() as conn:
        conn.execute(text(sql))

In [7]:
# Define server with port
bootstrap_servers = ['localhost:9092']

# Define topic name from where the message will recieve
topicName = f'DBCustomers.{database_source}.dbo.{table_name}'

# Initialize consumer variable
consumer = KafkaConsumer (topicName, 
                          bootstrap_servers = bootstrap_servers,
                          enable_auto_commit=True)

# Read and print message from consumer
for msg in consumer:
    try:
        #Read data from comsumer
        data = json.loads(msg.value)
        df = read_json_to_df(data)
        #Take change data
        df = pd.DataFrame([df.loc[0,'payload']])
        #Check action
        if (df.loc[0,'before'] == None):
            insert_data(df)
        elif (df.loc[0,'after'] == None):
            delete_data(df)
        else:
            update_data(df)
    #Temporary use try except to avoid bug
    except TypeError:
        time.sleep(0.001)

Update ID 1 from CustomerDetails
Update ID 6 from CustomerDetails
Delete ID 19 from CustomerDetails
Add ID 19 to CustomerDetails


KeyboardInterrupt: 